In [36]:
import argparse
import csv
import itertools
import logging
import time

import requests
import yaml
from bs4 import BeautifulSoup
from yaml.loader import FullLoader
logging.root.setLevel(logging.INFO)

In [37]:
# TODO: Load `env.yml` using yaml into an `ENV` variable
with open ('env.yml') as f:
    ENV= yaml.load(f, Loader=yaml.FullLoader)
ALL_CATEGORIES = ENV["CATEGORIES"] 
# TODO: In `env.yml`, add the other categories and their URLS
#Take Video OUt
ALL_CATEGORIES

{'HOME': 'https://www.bbc.com/pidgin',
 'MOST_POPULAR': 'https://www.bbc.com/pidgin/popular/read',
 'ENTERTAINMENT': 'https://www.bbc.com/pidgin/topics/cqywjyzk2vyt',
 'SPORT': 'https://www.bbc.com/pidgin/topics/cjgn7gv77vrt',
 'WORLD': 'https://www.bbc.com/pidgin/topics/c0823e52dd0t',
 'AFRICA': 'https://www.bbc.com/pidgin/topics/c404v061z85t',
 'NIGERIA': 'https://www.bbc.com/pidgin/topics/c2dwqd1zr92t'}

In [38]:
def get_parser() -> argparse.ArgumentParser:
    """
    parse command line arguments

    returns:
        parser - ArgumentParser object
    """
    parser = argparse.ArgumentParser(description="BBC Pidgin Scraper")
    parser.add_argument(
        "--output_file_name",
        type=str,
        default="bbc_pidgin_corpus.csv",
        help="Name of output file"
    )
    
    parser.add_argument(
    "--no_of_articles",
    type=int,
    default=100,
    help="No of Articles")
    
    parser.add_argument(
    "categories",
    type=str,
    default=all,
    help='Categories of News to be Scraped')
    
    parser.add_argument(
    "--time_delay",
    type=bool,
    default=False,
    help="Time Delay")
    
    # TODO: Add your other command line arguments!
    #   Be sure to include their types & any default values.

    return parser

In [39]:
def get_page_soup(url: str):
    
    """
    Makes a request to a url and creates a beautiful soup object from the response html

    input:
        :param url: input page url
    returns:
        - page_soup: beautiful soup object from the response html
    """

    response = requests.get(url)
    page_html=response.text
    category_page=BeautifulSoup(page_html, "html.parser")
    
    # TODO: Complete the function to parse and return the page

    return category_page

In [40]:
def get_valid_urls(category_page: BeautifulSoup):
    """
    Gets all valid urls from a category page

    input:
        :param: url: category_page
    returns:
        - valid_urls: list of all valid article urls on a given category page
    """
    all_urls = category_page.findAll("a")
    valid_article_urls = []
    for url in all_urls:
        href = url.get("href")
        if (href.startswith("/pidgin/tori") or  
           href.startswith ("/pidgin/world") or
           href.startswith ("/pidgin/sport") or
           href.startswith ("/pidgin/media")) and href[-1].isdigit() :
            valid_url="https://www.bbc.com" + href
            valid_article_urls.append(valid_url)
        # from a look at BBC pidgin's urls, they always begin with the following strings.
        # so we obtain valid article urls using these strings
    return list(set(valid_article_urls))


In [41]:
def get_urls(category_url: str, category: str, time_delay: bool) -> list:
    """
    Get valid urls from each category
    
    Input: 
         : Category url, Category, Time Delay: Bool
    
    Output:
          : Category Url for indicated Category
    """
    
    page_soup = get_page_soup(category_url)
    category_urls = get_valid_urls(page_soup)

    # get total number of pages for given category
    article_count_span = page_soup.find_all(
        "span", attrs={"class": "lx-pagination__page-number qa-pagination-total-page-number"}
    )
    # if there are multiple pages, get valid urls from each page
    # else just get the articles on the first page
    if article_count_span:
        total_article_count = int(article_count_span[0].text)
        logging.info(f"{total_article_count} pages found for {category}")
        logging.info(f"{len(category_urls)} urls in page 1 gotten for {category}")

        for count in range(1, total_article_count):
            
            # TODO: Use your `get_page_soup` and `get_valid_urls` functions
            #   to obtain valid urls from all pages
            page_soup=get_page_soup(category_url+f"/page/{count+1}")
            page_urls=get_valid_urls(page_soup)
            logging.info(f"{len(page_urls)} urls in page {count + 1} gotten for {category}")
            category_urls += page_urls
            if time_delay:
                time.sleep(10)
    else:
        logging.info(f"Only one page found for {category}. {len(category_urls)} urls gotten")

    return category_urls


In [42]:
article_urls=get_urls('https://www.bbc.com/pidgin/popular/read', 'Most_Popular', False)
print(article_urls)

INFO:root:Only one page found for Most_Popular. 10 urls gotten


['https://www.bbc.com/pidgin/tori-58929073', 'https://www.bbc.com/pidgin/tori-59021459', 'https://www.bbc.com/pidgin/tori-59003219', 'https://www.bbc.com/pidgin/tori-59020511', 'https://www.bbc.com/pidgin/tori-58977792', 'https://www.bbc.com/pidgin/tori-58993151', 'https://www.bbc.com/pidgin/tori-59020478', 'https://www.bbc.com/pidgin/tori-58957356', 'https://www.bbc.com/pidgin/tori-58991898', 'https://www.bbc.com/pidgin/world-59023596']


In [43]:
def get_article_data(article_url: str) -> tuple:
    """
    Obtains paragraphs texts and headlines input url article

    input:
        :param article_url: category_page
    returns:
        - headline: headline of url article
        - story_text: text of url article
        - article_url: input article url
    """
    page_soup = get_page_soup(article_url)
    # TODO: Locate the headline element
    headline = page_soup.find(
        "h1", attrs={"class": ENV["HEADLINE_SPAN_CLASS_A"]}
    )
    
    if headline:
        # TODO: Get the headline text from the element
        headline= headline.text.strip()
    else:
        headline=page_soup.find("h1", attrs={"class": ENV['HEADLINE_SPAN_CLASS_A']})
        
    story_text = " "
    story_div = page_soup.findAll("div", attrs = {"class": ENV['STORY_DIV_CLASS']})    # TODO: Locate all story divs
    if story_div:
        all_paragraphs = [div.findAll("p", recursive=False) for div in story_div]
        all_paragraphs = list(itertools.chain(*all_paragraphs))
        story_text = story_text.join(str(paragraph) for paragraph in all_paragraphs)
        story_text = BeautifulSoup(story_text, "html.parser").get_text()
    story_text = story_text if not story_text == " " else None

    return headline, story_text,article_url

In [44]:
article_data=get_article_data('https://www.bbc.com/pidgin/tori-59021459')
article_data

('Joy Onoh: How 24-year-old online seller die as she go make delivery',
 '"Afta I drink garri train Joy for University now wey she don finish to epp me, dem kill am" dis na wetin joy mama yarn BBC Pidgin on top di death of her pikin. 49 -year- old Grace Onoh wey be widow for 12 years  say all her hope don dabaru and her world don crash to pieces. " All our hope bin dey on her but as e dey now, I no know wia to start from for dis life again" she explain. Police on Wednesday 20th October, find Joy deadi bodi for ground around Federal Housing for North Bank area of Makurdi, afta she leave house to go deliver six turtleneck shirts to one customer for di area. Joy start di online business two months ago wit di business name Sparklight Collectibles to raise moni to pay her outstanding school fees for Benue State University. Who be Joy Israel Onoh? 24-year-old Joy Onoh come from Enugu state, for Nigeria southeast but her parents don live for Makurdi di Benue state capital for about 21 years n

In [46]:
def scrape(output_file_name: str, no_of_articles: int, categories: dict, time_delay: bool) -> None:
    """
    Main function for scraping and writing articles to file

    input:
        :param output_file_name: file name where output is saved
        :param no_of_articles: number of user specified articles to scrape
        :param category_urls: all articles in a category
    """
    logging.info("Writing articles to file...")

    with open(output_file_name, "w", encoding='utf-8') as csv_file:
        # TODO: Initialize a csv.DictWriter object
        headers=["headline", "text", "url"]
        writer=csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        #   Write the headers your CSV file will take
        story_num = 0

        for category, urls in categories.items():
            logging.info(f"Writing articles for {category} category...")
            for url in urls:
                # TODO: Get the headline, paragraphs and url using your `get_article_data` func
                headline, paragraphs, url = get_article_data(url)
                if paragraphs:
                    print (paragraphs)
                    # TODO: Write all the data to CSV
                    writer.writerow({headers[0]:headline, headers[1]: paragraphs, headers[2]: url})
                    story_num += 1
                    logging.info(f"Successfully wrote story number {story_num}")

                if story_num == no_of_articles:
                    logging.info(
                        f"Requested total number of articles {no_of_articles} reached"
                    )
                    logging.info(
                        f"Scraping done. A total of {no_of_articles} articles were scraped!"
                    )
                    return
            if time_delay:
                time.sleep(10)
                
    logging.info(
        f"Scraping done. A total of {story_num} articles were scraped!"
    )



In [47]:
# TODO: Based on this block, complete `get_parser`
if __name__ == "__main__":

    logging.info("--------------------------------------")
    logging.info("Starting scraping...")
    logging.info("--------------------------------------")

    # initialize parser
    parser = get_parser()
    params, unknown = parser.parse_known_args()

    # specify categories to scrape
    if params.categories != "all":
        categories = params.categories.upper().split(",")
#        categories = {category: ALL_CATEGORIES[category] for category in categories}
        for category in categories:
            categories=ALL_CATEGORIES
    else:
        categories = ALL_CATEGORIES[category]

    # get urls
    category_urls = {}
    for category, url in categories.items():
        logging.info(f"Getting all stories for {category}...")
        category_story_links = get_urls(url, category, params.time_delay)
        logging.info(f"{len(category_story_links)} stories found for {category} category")
        category_urls[category] = category_story_links
    # scrape and write to file
    scrape(params.output_file_name, params.no_of_articles, category_urls, params.time_delay)

INFO:root:--------------------------------------
INFO:root:Starting scraping...
INFO:root:--------------------------------------
INFO:root:Getting all stories for HOME...
INFO:root:Only one page found for HOME. 40 urls gotten
INFO:root:40 stories found for HOME category
INFO:root:Getting all stories for MOST_POPULAR...
INFO:root:Only one page found for MOST_POPULAR. 10 urls gotten
INFO:root:10 stories found for MOST_POPULAR category
INFO:root:Getting all stories for ENTERTAINMENT...
INFO:root:87 pages found for ENTERTAINMENT
INFO:root:10 urls in page 1 gotten for ENTERTAINMENT
INFO:root:9 urls in page 2 gotten for ENTERTAINMENT
INFO:root:10 urls in page 3 gotten for ENTERTAINMENT
INFO:root:10 urls in page 4 gotten for ENTERTAINMENT
INFO:root:8 urls in page 5 gotten for ENTERTAINMENT
INFO:root:9 urls in page 6 gotten for ENTERTAINMENT
INFO:root:9 urls in page 7 gotten for ENTERTAINMENT
INFO:root:10 urls in page 8 gotten for ENTERTAINMENT
INFO:root:10 urls in page 9 gotten for ENTERTAINM

INFO:root:9 urls in page 16 gotten for WORLD
INFO:root:6 urls in page 17 gotten for WORLD
INFO:root:10 urls in page 18 gotten for WORLD
INFO:root:10 urls in page 19 gotten for WORLD
INFO:root:10 urls in page 20 gotten for WORLD
INFO:root:9 urls in page 21 gotten for WORLD
INFO:root:8 urls in page 22 gotten for WORLD
INFO:root:10 urls in page 23 gotten for WORLD
INFO:root:8 urls in page 24 gotten for WORLD
INFO:root:10 urls in page 25 gotten for WORLD
INFO:root:10 urls in page 26 gotten for WORLD
INFO:root:8 urls in page 27 gotten for WORLD
INFO:root:10 urls in page 28 gotten for WORLD
INFO:root:10 urls in page 29 gotten for WORLD
INFO:root:9 urls in page 30 gotten for WORLD
INFO:root:10 urls in page 31 gotten for WORLD
INFO:root:10 urls in page 32 gotten for WORLD
INFO:root:10 urls in page 33 gotten for WORLD
INFO:root:9 urls in page 34 gotten for WORLD
INFO:root:10 urls in page 35 gotten for WORLD
INFO:root:10 urls in page 36 gotten for WORLD
INFO:root:10 urls in page 37 gotten for WO

INFO:root:10 urls in page 91 gotten for AFRICA
INFO:root:10 urls in page 92 gotten for AFRICA
INFO:root:4 urls in page 93 gotten for AFRICA
INFO:root:883 stories found for AFRICA category
INFO:root:Getting all stories for NIGERIA...
INFO:root:100 pages found for NIGERIA
INFO:root:10 urls in page 1 gotten for NIGERIA
INFO:root:10 urls in page 2 gotten for NIGERIA
INFO:root:9 urls in page 3 gotten for NIGERIA
INFO:root:8 urls in page 4 gotten for NIGERIA
INFO:root:8 urls in page 5 gotten for NIGERIA
INFO:root:9 urls in page 6 gotten for NIGERIA
INFO:root:10 urls in page 7 gotten for NIGERIA
INFO:root:10 urls in page 8 gotten for NIGERIA
INFO:root:9 urls in page 9 gotten for NIGERIA
INFO:root:10 urls in page 10 gotten for NIGERIA
INFO:root:10 urls in page 11 gotten for NIGERIA
INFO:root:9 urls in page 12 gotten for NIGERIA
INFO:root:10 urls in page 13 gotten for NIGERIA
INFO:root:10 urls in page 14 gotten for NIGERIA
INFO:root:10 urls in page 15 gotten for NIGERIA
INFO:root:10 urls in pag

Ground go shake on Sunday October 24, 2021 wen Manchester United jam Liverpool for Old Trafford for Premier League. Di two teams dey come into dis game afta dia midweek Champions League victories. Liverpool beat Atletico Madrid 3-2 for Spain and United beat Atalanta wit di same score line. Dis na everything you need to know ahead of di league match between di two most successful clubs for England. You see Manchester United performance wey dem come back from two goals  down against Atalanta and win 3-2 on Wednesday na di summary of how di team be under dia manager Ole Gunnar Solskjaer.  Dem fit play badly for some games and di next minute dem dey hard to stop. Dat na wetin BBC football sabi pesin Mark Lawrenson tok  If to say United lose dat game den e for be bonus for Liverpool but, instead di two teams moral dey high going into di weekend match. Although United fit dey under more pressure to get positive result, because dem don drop points for each of dia past three league games, but 

INFO:root:Successfully wrote story number 2


...

Fans of Big Brother Naija reality television show don react to Sunday eviction wey see Yousef and Sasky leave di Shine Ya House game. Tori be say five housemates bin dey up for possible eviction on Sunday dem be, Cross, Emmanuel, Yousef, Sakay and Angel. Di fans save Cross, Emmanuel and Angel. Yousef and Saskay na di fifteenth and sixteenth housemates to chop eviction dis season. Afta dem get di least amount of votes from di fans of di show. But some fans no dey happy say Angel avoid eviction yesterday. Na Yousef first comot from di house afta di host Ebuka Obi Uchendu tell am to leave di house As him join Ebuka on stage, Yousef say im no dey disappointed about im eviction. Rather im dey, grateful say im make to di house till dis moment. Im describe im stay for di game as "just sliding through" and add say e enjoy himself wella. For im future plans Yousef tell Ebuka say im dey look forward to endorsements and acting movies. Afta Yousef Saskay na di next to leave di house. Dis na wetin 

INFO:root:Successfully wrote story number 99


Di versatile Ivorian Laetitia Ky break Guinness World Record of pesin wey fit skip fastest wit im own hair in 30 seconds. Guinness World Records, on Thursday reveal di latest record breakers for year 2022 edition of di popular book. As usual, di annual publication feature pipo from ogbonge categories. From dog and cat wey fit ride scooter faster, to di world tallest teenager, pesin wey fit run pass on im hands, Pesin wey fit skip wit im own hair in 30secs and many odas. Di hairstyles Laetitia create fit take from 20 minutes up to 3 hours to finish and na different kind of style and leghts. Di 25-year-old Laetitia bin skip a rope wey she make out of her hair for 30 seconds.   She bin combine her hair-sculpting abilities plus athletic skills, and wit di help of extensions braided wit her natural hair, Laetitia Ky dey potry herself as a feminist artist wey dey create sculptures from her hair. She come from Abidjan, Ivory Coast. Inspired by di pre-colonial tradition of women for African cu

INFO:root:Successfully wrote story number 100
INFO:root:Requested total number of articles 100 reached
INFO:root:Scraping done. A total of 100 articles were scraped!


Saskay and Yousef chop eviction on Sunday night from Big Brother Naija "Shine Ya Eye" TV reality show. Saskay and Yousef fall out as di first two housemates to be evicted for di night. Angel, Cross and Emmanuel also dey nominated for possible eviction from Big Brother Naija season six for di night. At least one or two if not three of these five housemates fit comot from di "Shine Ya Eye" edition on Sunday 19 September, 2021. Housemates wey survive Ebuka Uchendu live eviction show on Sunday night go still get chance to win di 90 million naira [about $218,596USD] star prize. As 29-year-old Yousef step out of di house, e tell Ebuka for di live show say im dey eye to do movies afta spending eight weeks for di TV reality. Saskay on her part reveal to Ebuka say she don already taya for di TV reality show. Di 22-year-old say setting up a retirement home for old pipo and her lip-gloss brand na im dey her mind after di show. Shortly after di live eviction show, Angel hala on her own say she sur